In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import random
from scipy import stats
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [2]:
# assign directory
ROOT_DIR = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
data_dir = ROOT_DIR + 'raw-data/pastis/full-pastis-grey'
 
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [3]:
image = Image.open(file_list[0]).convert('L')
first_image = pywt.wavedec2(image, 'db1')
layer_len = len(first_image)



In [4]:
#for k in range(len(file_list)):
layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
orientation = [0] * len(file_list) * (layer_len - 1) * 3 
data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
c = 0
for k in range(len(file_list)):
    image = Image.open(file_list[k]).convert('L')
    image = np.array(image)
   
    transformed = pywt.wavedec2(image, 'db1')
    direction_names = ['H', 'V', 'D']

    for i in range(1, layer_len): 
        for j in range(len(transformed[i])):
            
            arr = np.array(transformed[i][j])
            layer_arr[c] = i+1
            orientation[c] =  direction_names[j]
            data_arr[c] = arr.flatten()
            c += 1

In [5]:
df = pd.DataFrame()

df["Layer"] = layer_arr
df["Orientation"] = orientation
df["Data"] = data_arr
new_df = pd.DataFrame(columns=["Channel","Layer", "Orientation", "Data"])
for lo, sf in df.groupby(["Layer", "Orientation"])[["Data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
    new_df.loc[len(new_df.index)] = ["Red", lo[0], lo[1],  np.hstack(sf['Data'])]
new_df

,Channel,Layer,Orientation,Data
0,Red,2,D,"[1248.6328125000014, -42.25000000000071, 371.8..."
1,Red,2,H,"[2441.0703125000023, -310.8281249999998, -1756..."
2,Red,2,V,"[991.3515625000009, -326.2343750000009, 1312.3..."
3,Red,3,D,"[327.6718749999999, -135.84375000000003, -270...."
4,Red,3,H,"[23.7656249999998, -177.50000000000026, 685.28..."
5,Red,3,V,"[356.328125, -4.156249999999773, -200.68749999..."
6,Red,4,D,"[-220.56250000000014, 67.68750000000001, 195.6..."
7,Red,4,H,"[-488.6875000000001, 89.99999999999993, 51.687..."
8,Red,4,V,"[631.0625000000002, 263.50000000000045, 87.687..."
9,Red,5,D,"[-137.00000000000017, -30.625000000000036, -17..."


In [6]:
def convert_to_wavelet_basis(folder_dir, color, basis="db1"):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    #Setup df Dict
    image = Image.open(file_list[0]).convert('L')
    first_image = pywt.wavedec2(image, basis)
    layer_len = len(first_image)
    print(str(layer_len) + " layers being used")
    
    color_dict = {"Red":0, "Green":1, "Blue":2, "Gray":3, "Infrared": 4}
    c = color_dict[color]

    #Fill DF DICT
    layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    orientation = [0] * len(file_list) * (layer_len - 1) * 3 
    data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    cnt = 0
    for k in range(len(file_list)):
        if c >= 3:
            image = np.array(Image.open(file_list[k]).convert('L'))
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std
        else:
            image = np.array(Image.open(file_list[k]))[:,:,c]
    
        transformed = pywt.wavedec2(image, 'db1')
        direction_names = ['H', 'V', 'D']

        for i in range(1, layer_len): 
            for j in range(len(transformed[i])):
                
                arr = np.array(transformed[i][j])
                layer_arr[cnt] = i+1
                orientation[cnt] =  direction_names[j]
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["Layer"] = layer_arr
    df["Orientation"] = orientation
    df["Data"] = data_arr
    new_df = pd.DataFrame(columns=["Channel","Layer", "Orientation", "Data"])
    for lo, sf in df.groupby(["Layer", "Orientation"])[["Data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [color, lo[0], lo[1],  np.hstack(sf['Data'])]
    
    return new_df

In [7]:
gray_converted = convert_to_wavelet_basis(data_dir, "Gray")
gray_converted

8 layers being used


,Channel,Layer,Orientation,Data
0,Gray,2,D,"[34.3161194559995, -0.9562662414870378, 7.5601..."
1,Gray,2,H,"[67.08782566471602, -7.035134741827539, -35.71..."
2,Gray,2,V,"[27.245270427843863, -7.383832417162677, 26.68..."
3,Gray,3,D,"[9.005391410752578, -3.7333876746498946, -7.42..."
4,Gray,3,H,"[0.6531496035360611, -4.878224520821578, 18.83..."
5,Gray,3,V,"[9.792949841314798, -0.11422603191360303, -5.5..."
6,Gray,4,D,"[-6.061709272527944, 1.8602525197358366, 5.378..."
7,Gray,4,H,"[-13.430576594473214, 2.4734659542193915, 1.42..."
8,Gray,4,V,"[17.343462319273055, 7.241758654853442, 2.4099..."
9,Gray,5,D,"[-3.765164841422849, -0.841665498310765, -4.82..."
